In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import copy

from itertools import product
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.naive_bayes import GaussianNB # Import Naive Bayes Classifier
from sklearn.svm import SVC, LinearSVC # Import SVM Classifier
from sklearn.neighbors import KNeighborsClassifier # Import KNN Classifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn import tree
from pprint import pprint
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import ClusterCentroids, AllKNN
from imblearn.combine import SMOTEENN, SMOTETomek
from sklearn.model_selection import KFold, cross_val_score, StratifiedKFold
from prettytable import PrettyTable
from scipy import stats
from matplotlib.pyplot import pie, axis, show
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.feature_selection import VarianceThreshold, SelectKBest, mutual_info_classif, SelectFromModel, RFE
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score

from helpers import read_and_process_data, plot_pie_chart

random.seed(0)

In [2]:
data = read_and_process_data('dataset_diabetes/diabetic_data.csv')
data.head(5)

---Retreived data from dataset_diabetes/diabetic_data.csv
Number of records: 101766 Number of features: 50
+--------------------------+---------------------------+
|         Feature          | Number of Unique Elements |
+--------------------------+---------------------------+
|       encounter_id       |           101766          |
|       patient_nbr        |           71518           |
|           race           |             6             |
|          gender          |             3             |
|           age            |             10            |
|          weight          |             10            |
|    admission_type_id     |             8             |
| discharge_disposition_id |             26            |
|   admission_source_id    |             17            |
|     time_in_hospital     |             14            |
|        payer_code        |             18            |
|    medical_specialty     |             73            |
|    num_lab_procedures    |          

race    gender       age  admission_type_id  discharge_disposition_id  \
0  0.396047 -0.924958 -3.287658          -0.707351                 -0.524964   
1 -1.926883 -0.924958 -2.646293          -0.707351                 -0.524964   
2  0.396047  1.080697 -2.004928          -0.707351                 -0.524964   
3  0.396047  1.080697 -1.363563          -0.707351                 -0.524964   
4  0.396047  1.080697 -0.722198          -0.017799                 -0.524964   

   admission_source_id  time_in_hospital  medical_specialty  \
0             0.407501         -0.475073          -0.351534   
1             0.407501         -0.809166           0.138537   
2             0.407501         -0.809166          -0.160097   
3             0.407501         -1.143260          -0.373961   
4            -1.012969         -0.475073           0.204408   

   num_lab_procedures  num_procedures  ...  citoglipton   insulin  \
0            0.804171       -0.790556  ...          0.0  1.905345   
1           -1.630877        2.135806  ...          0.0 -0.477883   
2            0.043218       -0.205284  ...          0.0  1.905345   
3            0.398330       -0.790556  ...          0.0  0.713731   
4           -0.616274        2.721079  ...          0.0  0.713731   

   glyburide-metformin  glipizide-metformin  glimepiride-pioglitazone  \
0            -0.081685            -0.011416                 -0.003097   
1            -0.081685            -0.011416                 -0.003097   
2            -0.081685            -0.011416                 -0.003097   
3            -0.081685            -0.011416                 -0.003097   
4            -0.081685            -0.011416                 -0.003097   

   metformin-rosiglitazone  metformin-pioglitazone    change  diabetesMed  \
0                      0.0               -0.003097 -1.079375      0.54876   
1                      0.0               -0.003097  0.926081      0.54876   
2                      0.0               -0.003097 -1.079375      0.54876   
3                      0.0               -0.003097 -1.079375      0.54876   
4                      0.0               -0.003097  0.926081      0.54876   

   readmitted  
0         1.0  
1         2.0  
2         2.0  
3         2.0  
4         1.0  

[5 rows x 46 columns]

In [3]:
features = list(data.columns[0:1]) + list(data.columns[2:])
X = np.array(data[features]) # Features
y = np.array(data[data.columns[1]]).astype(int) # Target variable
print("Input data shape", X.shape, "Output data shape", y.shape)

Input data shape (98053, 45) Output data shape (98053,)


In [4]:
kfold = StratifiedKFold(10, random_state=False, shuffle=True)

folds = kfold.split(X, y)
folds = [(train, test) for train, test in folds]

In [5]:
estimator = SVR(kernel="linear")
selector = RFE(estimator, n_features_to_select=25, step=1)
selected = selector.fit(X[folds[0][1]], y[folds[0][1]])
print(selected.support_)
print(selected.ranking_)

[ True  True False  True  True  True  True  True  True  True  True  True
 False  True  True  True  True False  True  True False False False  True
 False  True  True False False  True  True  True False False False False
 False  True False False False False False  True False]
[ 1  1  5  1  1  1  1  1  1  1  1  1  6  1  1  1  1  9  1  1 13  3 16  1
 18  1  1 15 12  1  1  1  8  4 19 20  2  1  7 17 21 14 10  1 11]


In [6]:
selected_feat = []
for tak, (i, col) in zip(selected.support_, enumerate(data.columns[:-1])):
    if tak:
        print(col, i)
        selected_feat.append((col, i))

race 0
gender 1
admission_type_id 3
discharge_disposition_id 4
admission_source_id 5
time_in_hospital 6
medical_specialty 7
num_lab_procedures 8
num_procedures 9
num_medications 10
number_outpatient 11
number_inpatient 13
diag_1 14
diag_2 15
diag_3 16
max_glu_serum 18
A1Cresult 19
chlorpropamide 23
acetohexamide 25
glipizide 26
pioglitazone 29
rosiglitazone 30
acarbose 31
insulin 37
change 43


In [7]:
X_rfe = selector.transform(X)
X_rfe.shape

(98053, 25)

In [9]:
# # X_cs, y_cs = SMOTEENN(random_state=0).fit_resample(X_rfe, y)

# X_cs, y_cs = ADASYN(sampling_strategy='not majority', 
#                     n_neighbors=7, random_state=0).fit_resample(X, y)

# print('After over-sampling', X_cs.shape, y_cs.shape)

In [10]:
# X_cs, y_cs = AllKNN(sampling_strategy='all', n_neighbors=7).fit_resample(X_cs, y_cs)

# print('After combined-sampling', X_cs.shape, y_cs.shape)

In [11]:
np.save('npy_data_2/X_combinedSampling_25Features_task2.npy', X_rfe)
np.save('npy_data_2/y_combinedSampling_25Features_task2.npy', y)
np.save('npy_data_2/selected_features_25_task2.npy', selected_feat)